In [1]:
import numpy as np
import itertools


## method 1: testing for all permutation of path, then find the desired sum
## the method cannot process a large set of data
def q1method1(rows, cols, select_sum):
    if rows*cols == 0:
        print("Please enter a non-empty matrix size.")
        return 0
    mat = np.arange(1,rows+1).repeat(cols).reshape(rows,cols)
    empty_mat = np.zeros((rows, cols))
    empty_mat[0][0]= mat[0][0]

    # indexs of path: denotes 1 for right(R), 0 for down(D)
    paths = np.hstack((np.arange(1).repeat(rows-1),np.arange(1).repeat(cols-1)+1))
    for a in itertools.permutations(paths):
        paths = np.vstack((paths,a))
    paths = np.unique(paths,axis=0) # remove duplicates
    
    # walk the path
    summ = np.zeros(len(paths),dtype=np.int) # calculate the sum
    c = 0
    for b in paths:
        x, y = 0, 0 # use x and y to denote the current point
        for i in b:
            if i == 0: # go down
                x += 1
            else: 
                y += 1
            empty_mat[x][y] = mat[x][y]
        summ[c] = int(empty_mat.sum())
        #print(' Path:\n', empty_mat, '\n Sum:', summ[c]) # you can check by print
        
        empty_mat = np.zeros((rows, cols))
        empty_mat[0][0]= mat[0][0]
        c += 1
    
    # select paths
    paths = ','.join(map(str, paths))
    paths = paths.split(',') 
    dic = dict(zip(paths, summ)) # comput dictionary
    print(sorted(dic.items(), key=lambda x:x[1]) )

    select_dic = {k: v for k, v in dic.items() if v in select_sum}
    print('The respective operations are:',select_dic.items())
    return select_dic
    
        
# test run
result = q1method1(rows=4, cols=4, select_sum=[13,16,19])
## however, this method takes too much time to process for a large matrix

[('[1 1 1 0 0 0]', 13), ('[1 1 0 1 0 0]', 14), ('[1 0 1 1 0 0]', 15), ('[1 1 0 0 1 0]', 15), ('[0 1 1 1 0 0]', 16), ('[1 0 1 0 1 0]', 16), ('[1 1 0 0 0 1]', 16), ('[0 1 1 0 1 0]', 17), ('[1 0 0 1 1 0]', 17), ('[1 0 1 0 0 1]', 17), ('[0 1 0 1 1 0]', 18), ('[0 1 1 0 0 1]', 18), ('[1 0 0 1 0 1]', 18), ('[0 0 1 1 1 0]', 19), ('[0 1 0 1 0 1]', 19), ('[1 0 0 0 1 1]', 19), ('[0 0 1 1 0 1]', 20), ('[0 1 0 0 1 1]', 20), ('[0 0 1 0 1 1]', 21), ('[0 0 0 1 1 1]', 22)]
The respective operations are: dict_items([('[0 0 1 1 1 0]', 19), ('[0 1 0 1 0 1]', 19), ('[0 1 1 1 0 0]', 16), ('[1 0 0 0 1 1]', 19), ('[1 0 1 0 1 0]', 16), ('[1 1 0 0 0 1]', 16), ('[1 1 1 0 0 0]', 13)])


Some explaination:
In the case for 4 by 4 matrix, from the result we can see the sum gradually increase by 1 or even 0 from 1*3+2+3+4=13(going right till reach the end then go down) to 1+2+3+4*3=22(going down till reach the end then go right).
Following the same logic, in the case of m=9, n=9 matrix, the summ is ranging from 1*9+2+3+...+9=53(RRRRRRRRDDDDDDDD) to 1+2+3+...+8+9*9=117(DDDDDDDDRRRRRRRR).
In addition, according to the property of this type of matrix, if the path go down from 1 in ith value, then going down till the bottom, the total sum will increase by (9-1)*(9-i) (i.e. RRRRRRRDDDDDDDDR), when the final right turn set as higher position, it will decrease by 1 (i.e. RRRRRRRDDDDDDDRD). 


In [2]:
# Given that the method 1 required huge computation and calculation, a new method is designed according to the above pattern
import numpy as np

data = open('output_question_1.txt','w+')
    
def q1method2(rows, cols, select_sum):
    base = "R"*(cols-1) + "D"*(rows-1) # 1 go right and 0 go down
    n = cols+rows-2
    
    paths = np.hstack((np.arange(1).repeat(cols-1),np.arange(1).repeat(rows-1)+1))
    minpaths = np.sum(np.cumsum(np.hstack((1,paths))),dtype=np.int64)
    index = minpaths
    result = {index:[base]}
    while (1):
        for answer in result[index]:
            for i in range(n-1):
                if answer[i] == "R":
                    if answer[i+1] == "D":
                        if i < n-2:
                            newanswer = answer[:i] + "DR" + answer[i+2:]
                        else:
                            newanswer = answer[:i] + "DR" 

                        if (index+1) not in result.keys():
                            result[index+1] = [newanswer]
                        else:
                            result[index+1] = result[index+1] + [newanswer]
        if index+1 not in result.keys():
            break                  
        result[index+1] = list(set(result[index+1]))
        if index in select_sum:
            for j in result[index]:
                print(str(index)+' '+str(j),file=data)
        index += 1
        if index> max(select_sum): break
    return result, minpaths

#result,minpaths = q1method2(rows=4, cols=4, select_sum=[13,16])

In [3]:
resulta = q1method2(rows=9, cols=9, select_sum=[65,72,91,110])
#data = open('output_question_1.txt','w+')
#for a,b in resulta:
#    print(str(b)+' '+str(a), file = data)

In [4]:
# question b: 
# since D...DR...R will have the largest sum and R...RD...D will have the smallest sum
# the sum will be within 4050144999 to 14999950000, while 87127231192>14999950000, so there is no path for it
# since path D...D(10527)R...R(89472),D...D(89999) has sum 5994850344
#       path D...D(10528)R...R(89471),D...D(89999) has sum 5995029815
# to reduce combination, we limited the permutaion to the last 189998-10527 moves
    
def q1method2modified(rows, cols, select_sum):
    base = "D"*10527+"R"*(cols-1) + "D"*(rows-10527-1) # 1 go right and 0 go down
    n = cols+rows-2
    
    paths = np.hstack(np.arange(1).repeat(10527)+1,np.arange(1).repeat(cols-1),np.arange(1).repeat(rows-10527-1)+1)
    minpaths = np.sum(np.cumsum(np.hstack((1,paths))),dtype=np.int64)
    index = minpaths
    result = {index:[base]}
    while (1):
        for answer in result[index]:
            for i in range(n-1):
                if answer[i] == "R":
                    if answer[i+1] == "D":
                        if i < n-2:
                            newanswer = answer[:i] + "DR" + answer[i+2:]
                        else:
                            newanswer = answer[:i] + "DR" 

                        if (index+1) not in result.keys():
                            result[index+1] = [newanswer]
                        else:
                            result[index+1] = result[index+1] + [newanswer]
        if index+1 not in result.keys():
            break                  
        result[index+1] = list(set(result[index+1]))
        if index in select_sum:
            for j in result[index+1]:
                print(str(index)+' '+str(j),file=data)
        index += 1
        if index> max(select_sum): break
    return result, minpaths


In [ ]:
print('\n', file=data)

resultb = q1method2(rows=90000, cols=100000, select_sum=[5994891682,87127231192])

In [ ]:
# above two methods are applicable for question 2, 
# but due to limited calculation power, the result cannot yet been computed
# By mathematical calculation, we is clear that:
# There is no path for 87127231192;
# D...D(10527)R...R(89471)D...D(48661)RD...D(89998) 
# and D...D(10528)R...R(89471),D...D(39134)RD...DRD...D(89997) 
# are paths for 5994891682